# How to roll your own scikit-learn compatible algorithm


First off you need to implement you algorithm as a class and provide named parameters with default values in the
constructor

ex) `def __init__(self, max_depth=5, min_samples_split=20)`

then you will need to provide the following methods at the bare minimum:

```
def fit(self, X, y)
    """train your model"""

def predict(self, X)
    """to make use of your model"""

def score(self, X, y, sample_weight=None)
    """for evaluating your model's accuracy"""
```  

However, this will only get you so far, and primarily provides the same "feel" as a scikit learn estimator.  to make it compatible with more things, you should also implement:

```
def get_params(self, deep=True)
    """returns a dict of the current parameter values.
    
    It should accept the named parameter **deep** with a default value of **True** for compatability.
    In most cases, you will ignore this value. If you have sub-estimators, this value signifies that the user
    wants these params exposed as well
    """

def set_params(self, **params)
    """Allows parameter values passed to the constructor to be modified"""
```

## What I've learned since my presentation

I simply subclassed from `object`, and implemented the above interface. If I had instead subclassed from sklearn's  `base.BaseEstimator`, I would have inherited:

* `get_params()`  - BaseEstimator introspects your __init__() and returns a dict of your named params mapped to their values
* `set_params()`  - this method is always the same

It also implements `__repr__()`, which makes use of `get_params()`, so you don't need to implement this method either


If I had made use of `base.ClassifierMixin` I would have inherited:
* `score()` - I wrote my own custom code for calculating accuracy.  this calls `sklearn.metrics.accuracy_score()` so you don't need to roll your own

Also it automatically sets the following attributes for you:
* `estimator_type_ = "classifier"`

If instead you're implementing a regression estimator, you could make use of `base.RegressorMixin` which provides
* `score()` which uses sklearn.metrics.r2_score

and sets:
* `estimator_type_ = "regressor"`


Anyway, as you can see simply subclassing 'object' and providing the public interface described abovegot me really far.  

If, however, I'd have subclassed from `base.BaseEstimator` and made use of either the `base.BaseClassifier` or `base.BaseRegresion` mixins, I could have reduced some of the boilerplate necessary to make my code scikit-learn compatible, and spendt more time on implementing the actual algoritm, meaning I'd have only needed to provide:
* `fit()`
* `predict()`



In [1]:
# installs 'PghML' in sys.path (if it isn't there already)
from config_notebook import setup_pgh_ml_path
setup_pgh_ml_path()

# loader function for my dataset
from pgh_ml_py.datasets import load_banknote_authentication
# my decision tree implementation
from pgh_ml_py.sklearn_compat.tree.cart_decision_tree import CartDecisionTreeClassifier, display_tree

# useful sklearn functions/Classes which we wish to be able to leverage (the point of making our code compatible)
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
# other dependencies
import numpy as np
import pandas as pd

In [2]:
# my custom dataset loader function. typically you'll have it return some fields using the conventions
# data -> the features matrix
# target -> the labels vector
# in addition, I'm also returning 'dataframe' which is the original pandas dataframe I loaded, so we can analyze
# the data as well
dataset = load_banknote_authentication()

In [3]:
df = dataset.dataframe
X = dataset.data
y = dataset.target

In [4]:
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [5]:
print train_X.shape
print train_y.shape
print test_X.shape
print test_y.shape

(1029, 4)
(1029,)
(343, 4)
(343,)


In [6]:
clf = CartDecisionTreeClassifier()

In [7]:
clf.fit(train_X, train_y)

CartDecisionTreeClassifier(criterion=u'gini', max_depth=5,
              min_samples_split=20, splitter=u'best')

### Following sklearn's conventions for decision trees, my implementation's fit method sets the following 2 attributes:

* `clf.tree_`  - the underlying representation of the decision tree

* `clf.classes_` - the set of unique classes in y



### Here I call my ad_hoc function, display_tree(), which understands my decision trees representation makes use of these attributes

In [8]:
display_tree(clf.tree_, clf.classes_)

if feat[0] <= -1.389: (impurity: 3.995 num_samples: 1029 [545 484])
T-> if feat[0] <= -1.389: (impurity: 3.987 num_samples: 327 [ 36 291])
  T-> 1
  F-> 1
F-> if feat[0] <= -1.119: (impurity: 3.993 num_samples: 702 [509 193])
  T-> if feat[0] <= -1.119: (impurity: 3.798 num_samples: 23 [ 9 14])
    T-> 1
    F-> 1
  F-> if feat[0] <= 0.191: (impurity: 3.993 num_samples: 679 [500 179])
    T-> if feat[0] <= 0.191: (impurity: 3.971 num_samples: 147 [ 42 105])
      T-> 1
      F-> 1
    F-> if feat[0] <= 3.999: (impurity: 3.992 num_samples: 532 [458  74])
      T-> if feat[0] <= 1.208: (impurity: 3.989 num_samples: 413 [339  74])
        T-> 0
        F-> 0
      F-> if feat[0] <= 4.340: (impurity: 3.965 num_samples: 119 [119])
        T-> 0
        F-> 0


## Ok, great, but let's actually try and do something with my tree.  Let's call predict() on it

In [9]:
clf.predict(test_X[0])

/home/ckot/PghML/venv/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[0]

## yikes!

Ehat's happening here ??? As you can see, it *works* but spits out an ugly deprecation warning

Sklearn classfiers predict methods expect an **array** of rows, so if we're passing in a single row of data we simply need to pass it as [row]


In [10]:
clf.predict([test_X[0]])

[0]

### And let's see how accurate my tree is by passing in full test data set

In [11]:
clf.score(test_X, test_y)

0.8571428571428571

### Ok. 
That's fine for demonstrating how to fit/predict/score a single tree, but let's do a cross validation with 5 folds


In [12]:
cross_val_score(clf, dataset.data, dataset.target, cv=5)

array([ 0.68      ,  0.55636364,  0.82846715,  0.81386861,  0.80291971])

### Hmm. something looks **very** wrong here,  the original code scored ~80%.  Did I break something in my refactoring?  Lets take a look at the data (like I **should** have prior to doing anything)

In [13]:
df.head()

,variance,skewness,curtosis,entropy,label
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [14]:
df.tail()

,variance,skewness,curtosis,entropy,label
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1
1371,-2.54190,-0.65804,2.6842,1.19520,1


## Ok,  I *think* I'm seeing a pattern. Let's print out some more of the dataset to make sure I'm not hallucinating

In [15]:
df

,variance,skewness,curtosis,entropy,label
0,3.621600,8.66610,-2.807300,-0.446990,0
1,4.545900,8.16740,-2.458600,-1.462100,0
2,3.866000,-2.63830,1.924200,0.106450,0
3,3.456600,9.52280,-4.011200,-3.594400,0
4,0.329240,-4.45520,4.571800,-0.988800,0
5,4.368400,9.67180,-3.960600,-3.162500,0
6,3.591200,3.01290,0.728880,0.564210,0
7,2.092200,-6.81000,8.463600,-0.602160,0
8,3.203200,5.75880,-0.753450,-0.612510,0
9,1.535600,9.17720,-2.271800,-0.735350,0


### Here's the problem with what I did

It turns out that my original function which created k-folds was randomizing (shuffling) the order of records, but that's not happening here

As you can see, **all** of the rows labeled **0** are in the **1st half** of the dataset while all the rows labeled **1** are in the **2nd half** of the dataset.  

By default, if you simply pass in an int for the cv param it uses KFold which doesn't deal with this. 

### Let's make use of StratifiedKFold instead to make sure that all of our folds have the classes balanced

In [16]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=np.random.RandomState(1))

In [17]:
cross_val_score(clf, dataset.data, dataset.target, cv=cv)

array([ 0.55458515,  0.74179431,  0.55579869])

### Ok, great!  These values are pretty much in sync with the original blog post I based this off of.  That's a relief - I didn't break anything in all of my refactoring.

So far, I've simply made use of my class using it's default values of max_depth=5 and min_samples_split=20

### Let's make use of sklearn's GridSearchCV to try automatically optimize values for these parameters. 

Of course, this can take a while, so I'll keep the ranges of values to a reasonable size

In [18]:
parameters = {'max_depth': range(3, 6), 'min_samples_split': range(10, 26, 5)}

In [19]:
dt = CartDecisionTreeClassifier()

In [20]:
clf = GridSearchCV(dt, parameters, cv=cv, verbose=True)
print clf

GridSearchCV(cv=StratifiedKFold(n_splits=3,
        random_state=<mtrand.RandomState object at 0x7fe67c4fef00>,
        shuffle=True),
       error_score='raise',
       estimator=CartDecisionTreeClassifier(criterion=u'gini', max_depth=5,
              min_samples_split=20, splitter=u'best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [10, 15, 20, 25], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)


In [21]:
clf.fit(X, y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  2.5min finished


GridSearchCV(cv=StratifiedKFold(n_splits=3,
        random_state=<mtrand.RandomState object at 0x7fe67c4fef00>,
        shuffle=True),
       error_score='raise',
       estimator=CartDecisionTreeClassifier(criterion=u'gini', max_depth=5,
              min_samples_split=20, splitter=u'best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [10, 15, 20, 25], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

### After running fit(), the GridSearchCV has a bunch of attributes set.  The ones I found most useful were:
* cv\_results\_      - lots of details which can be imported into pandas as a dataframe
* best\_score\_      - score of the best result
* best\_params\_     - dict of the best parameter values discovered
* best\_estimator\_  - the best estimator object (useful as you can inspect it)

In [22]:
df2 = pd.DataFrame(clf.cv_results_)

In [23]:
df2

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,4.104398,0.001884,0.654519,0.666978,3,10,"{u'min_samples_split': 10, u'max_depth': 3}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,0.853014,0.000172,0.125806,0.134124
1,4.203046,0.001887,0.654519,0.666978,3,15,"{u'min_samples_split': 15, u'max_depth': 3}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,0.685075,0.000150,0.125806,0.134124
2,4.162250,0.001863,0.654519,0.666978,3,20,"{u'min_samples_split': 20, u'max_depth': 3}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,0.655186,0.000148,0.125806,0.134124
3,4.009429,0.001956,0.654519,0.666978,3,25,"{u'min_samples_split': 25, u'max_depth': 3}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,0.744992,0.000122,0.125806,0.134124
4,4.367380,0.002060,0.654519,0.666978,4,10,"{u'min_samples_split': 10, u'max_depth': 4}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.087559,0.000306,0.125806,0.134124
5,4.259733,0.002049,0.654519,0.666978,4,15,"{u'min_samples_split': 15, u'max_depth': 4}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.120363,0.000362,0.125806,0.134124
6,4.246996,0.002032,0.654519,0.666978,4,20,"{u'min_samples_split': 20, u'max_depth': 4}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.121069,0.000361,0.125806,0.134124
7,4.240975,0.002047,0.654519,0.666978,4,25,"{u'min_samples_split': 25, u'max_depth': 4}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.106430,0.000331,0.125806,0.134124
8,4.260208,0.002059,0.654519,0.666978,5,10,"{u'min_samples_split': 10, u'max_depth': 5}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.124908,0.000329,0.125806,0.134124
9,4.247306,0.002039,0.654519,0.666978,5,15,"{u'min_samples_split': 15, u'max_depth': 5}",1,0.831878,0.85558,0.575492,0.590164,0.555799,0.555191,1.110683,0.000314,0.125806,0.134124


In [24]:
print """
best score: %f
best params: %s
""" % (clf.best_score_, clf.best_params_)


best score: 0.654519
best params: {'min_samples_split': 10, 'max_depth': 3}



In [25]:
best_tree = clf.best_estimator_

In [26]:
display_tree(best_tree.tree_, best_tree.classes_)

if feat[0] <= 4.676: (impurity: 3.996 num_samples: 1372 [762 610])
T-> if feat[0] <= 4.368: (impurity: 3.996 num_samples: 1301 [691 610])
  T-> if feat[0] <= 0.936: (impurity: 3.996 num_samples: 1273 [663 610])
    T-> 1
    F-> 0
  F-> if feat[0] <= 4.546: (impurity: 3.849 num_samples: 28 [28])
    T-> 0
    F-> 0
F-> if feat[0] <= 5.402: (impurity: 3.942 num_samples: 71 [71])
  T-> if feat[0] <= 5.242: (impurity: 3.915 num_samples: 49 [49])
    T-> 0
    F-> 0
  F-> if feat[0] <= 5.787: (impurity: 3.810 num_samples: 22 [22])
    T-> 0
    F-> 0


## Going Forward

Decision Trees in sklearn provide the following attributes
* `classes_` - I implemented this one. simply the unique set of labels 


* `tree_` - I implemented this as well, simply provides access to the data structure which represents the tree


* `n_features_` -  I *believe* this should be the number of columns in X, or simply `X.shape[1]`


* `n_outputs_` - I *believe* this represents how many columns are in y, which is typically 1, but is computed as `y.ndim`  Perhaps this is more useful when doing multi-label stuff


* `n_classes_` - I *believe* this should simply be  `len(classes_)`


* `max_features_` - how many features were considered when determining the best splits

Also they provide more named-parameters in their constructors. I'm currently providing:
* `max_depth`


* `min_samples_split`

while sklearn decision trees also provide the following named parameters in their constructors:

* For classifiers:
    * `criterion="gini"`  the other choice being "entropy". I could add support for using this as my metric for selecting the best split
* For regressors:
    * `criterion="mse"`  the other choices being "friedman_mse", and "mae"
 
 
* `splitter="best"`  the other choice being "random". My implementation is "best".  I *believe* "random" would be useful for generating numerous trees for Random Forests


* `max_features` defaults to the number of features, different values determine how many features should be considered when trying to determine the best split


* `min_samples_leaf` the minimum number of samples required to represent a leaf node. I'm guessing this is used to prune the parent node and replace it with a leaf


* `max_leaf_nodes=None` - grows tree with 'max_leaf_nodes' in best-first fashion. best nodes are defined as relative reduction in impurity. If none, unlimited number of leaf nodes


* `min_weight_fraction_leaf=0` - minimum weighted fraction of the sum total of weights (of all input samples) required to be a leaf node. 0 means samples all have the same weight


* `class_weight=None`  if 'None' all classes have weight of 1, if "balanced" each class's weight is inversely proportional to class frequencies in the y. can also be a dict of {class_label: weight, ...} or for multi-label a list of dicts in this format


* `random_state=None` - scikit-learn always randomizes the feature indices to use when calculating splits, even if max_features is the same as n_features_.  this parameter simply gives you control to make multiple runs deterministic


* `min_impurity_split=1e-7` threshold to prevent splits if a node's impurity is below, and instead generate a leaf node


* `presort=False` whether to presort the data to attempt to speed up finding the best splits. A setting of True may slow down fitting of large datasets (if max_depth is too high), but may speed up things up for small datasets 


Perhaps, by incorporating such hyper-parameters into my algorithm, and thus being able to search for the optimal values, I improve the trees I generate.  

Again, my purpose isn't to re-implement scikit-learn's decision trees, but rather implement an algoritm which is compatable with sklearn.  Figuring out what additional hyper-parameters are relevant to decision trees would simply allow me to explore improving the algorithm.  Perhaps other hyper-parameters could be found from different implementations or my own ideas.


## References

Original blog post I got the algorithm from: http://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

sklearn documentation on rolling your own estimator: http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator
